#Webscrapping Major Football Leagues Using Selenium. Importing Required libraries 

In [2]:
#importing necessary libraries #
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.service import Service
#excel dataframe
import xlsxwriter
#import By for element types
from selenium.webdriver.common.by import By
#import alert
from selenium.webdriver.common.alert import Alert

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException

In [ ]:
Specifying the objects to obtain specifying the Date, Home Team Name, Home Team Half Time Score, Home Team Full Time Score, Away Team Name, Away Team Half Time Score and Away Team Full Time Score. We will start from the premier league, note that we will use the same function for all the other data sets.

In [6]:
#adding a scroll element to the table to add more matches
# Get the element that triggers the scrolling action
#scroll_element = driver.find_element(By.CLASS_NAME, "event__more--static")

In [3]:
import time

In [4]:
#get working directory so we know where to get our chromDriver from
import os
# Get and print the current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\John\Webscrapping-Football-Leagues-Scores-Using-Selenium-in-Python


In [20]:
#with new version of chrome the steps to get our chrome web driver have changed
service_obj = Service("C:\\Users\\John\chromedriver-win32\chromedriver.exe")
driver = webdriver.Chrome(service=service_obj)

In [21]:
# english premier league
#putting an object to have the list
element_list=[]

driver.get("https://www.flashscore.co.ke/football/germany/bundesliga/results/")

#scroll_element = driver.find_element(By.CLASS_NAME, "event__more--static")


date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
    
    #driver.execute_script("arguments[0].click();",scroll_element)
    
print(element_list[:3])

#closing the driver
driver.close()

[['14.01. 19:30', 'B. Monchengladbach', '(2)', '3', 'Stuttgart', '(0)', '1'], ['14.01. 17:30', 'Bochum', '(0)', '1', 'Werder Bremen', '(0)', '1'], ['13.01. 20:30', 'Darmstadt', '(0)', '0', 'Dortmund', '(1)', '3']]


In [3]:
#turning the element list into a data frame using pandas
#i will go to the steps of dealing with each row/column
#then maybe finally write a function that does that cleaning in
#the above data frame does not look very good

In [12]:
len(element_list) #now I will extract the data with a scrollable element

109

Converting the list of elements obtained into a data frame using Pandas Data Frame

In [22]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)

In [23]:
df2=clean_data(df)

<ipython-input-14-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-14-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-14-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-14-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a

In [24]:
df2.to_csv('bundesliga-saturday.csv',index=False)

As seen above the data needs some cleaning. First we remove the brackets from the half time scores for both the home team and the away team. Then we deal with separating the date of the match and the time. Also we need to convert the Full Time score into an integer, to represent the number of goals as an integer. So we will make one function to do it all at once.

In [14]:
#making one data cleaning function for the all data sets
def clean_data(df):
    #removing the brackets in Half Time Scores
    df['Home Team HT']=df['Home Team HT'].str.replace('(','')
    df['Home Team HT']=df['Home Team HT'].str.replace(')','')

    df['Away Team HT']=df['Away Team HT'].str.replace('(','')
    df['Away Team HT']=df['Away Team HT'].str.replace(')','')
    
    
    #converting columns to integers
    df['Home Team HT']=df['Home Team HT'].astype(int)
    df['Away Team HT']=df['Away Team HT'].astype(int)
    df['Home Team FT']=df['Home Team FT'].astype(int)
    df['Away Team FT']=df['Away Team FT'].astype(int)
    
    
    #converting date column into date time
    d=df['Date'].str.split(' ',expand=True) #expanding by space
    #labelling d
    d.rename(columns={0:'date',
                 1:'time'},inplace=True)
    #adding 2024 to date column
    d['date']=d['date'] + '2024'
    #turning into date and time types
    d['date']=pd.to_datetime(d['date'],format='%d.%m.%Y')

    #converting time column into time
    #d['time']=pd.to_datetime(d['time'],format='%H:%M')

    #merging with the final data set
    df=pd.merge(df,d,on=df.index)
    
    df.drop(columns=['key_0','Date'],inplace=True)
    
    #creating Columns of No of Goals
    df['Total HT Goals']=df['Home Team HT'] + df['Away Team HT']
    df['Total FT Goals']=df['Home Team FT'] + df['Away Team FT']
    df['FT-HT Goals']=df['Total FT Goals'] - df['Total HT Goals']
    
    #save csv
    return(df)

In [5]:
df2=clean_data(df)

<ipython-input-4-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-4-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-4-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-4-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [6]:
df2.to_csv('greece ligue 2.csv',index=False)

In [6]:
#england southern league central division
# english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/england/isthmian-league-premier-division/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-6-7abce5e9c5c9>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['20.12. 22:45', 'Billericay', '(0)', '2', 'Aveley', '(0)', '0'], ['20.12. 22:45', "Bishop's Stortford", '(0)', '1', 'Potters Bar', '(0)', '1']]


In [7]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)

In [8]:
df2=clean_data(df)

df2.to_csv('england isthmian.csv',index=False)

<ipython-input-2-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-2-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [9]:
#la liga
"https://www.livesport.com/en/soccer/spain/laliga/results/"
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/spain/laliga/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-9-085aeb92896f>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['04.11. 23:00', 'Girona', '(0)', '2', 'Ath Bilbao', '(0)', '1'], ['31.10. 23:00', 'Elche', '(0)', '0', 'Getafe', '(0)', '1']]


In [10]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)

In [12]:
df2.to_csv('premier league3.csv',index=False)

In [11]:
df2=clean_data(df)

df2.to_csv('la liga.csv',index=False)

<ipython-input-5-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-5-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-5-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-5-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [12]:
#

In [13]:
#ligue 1
'https://www.livesport.com/en/soccer/france/ligue-1/results/'
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/france/ligue-1/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-13-df2607815c8b>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['04.11. 23:00', 'Troyes', '(1)', '1', 'Auxerre', '(0)', '1'], ['30.10. 22:45', 'Lyon', '(0)', '1', 'Lille', '(0)', '0']]


In [14]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)


In [15]:
df2=clean_data(df)

df2.to_csv('ligue one.csv',index=False)

<ipython-input-5-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-5-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-5-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-5-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [6]:
#bundesliga
"https://www.livesport.com/en/soccer/germany/bundesliga/results/"
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/germany/bundesliga/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-6-a1c3eb007103>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['11.11. 22:30', 'B. Monchengladbach', '(3)', '4', 'Dortmund', '(2)', '2'], ['09.11. 22:30', 'Eintracht Frankfurt', '(3)', '4', 'Hoffenheim', '(1)', '2']]


In [7]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)


In [8]:
df2=clean_data(df)

df2.to_csv('bundesliga.csv',index=False)

<ipython-input-2-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-2-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [19]:
#seria A
"https://www.livesport.com/en/soccer/italy/serie-a/results/"
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/italy/serie-a/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-19-513b8217ffe8>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['04.11. 22:45', 'Udinese', '(0)', '1', 'Lecce', '(1)', '1'], ['31.10. 22:45', 'Monza', '(0)', '1', 'Bologna', '(0)', '2']]


In [20]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)


In [21]:
df2=clean_data(df)

df2.to_csv('seriea.csv',index=False)

<ipython-input-5-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-5-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-5-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-5-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [3]:
#Europa League
"https://www.livesport.com/en/soccer/europe/europa-league/results/"
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/europe/europa-league/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-3-9ee5893c6c8f>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['27.10. 22:00', 'Crvena zvezda', '(1)', '2', 'Trabzonspor', '(1)', '1'], ['27.10. 22:00', 'Ferencvaros', '(0)', '1', 'Monaco', '(1)', '1']]


In [4]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)


In [5]:
df2=clean_data(df)

df2.to_csv('europa.csv',index=False)

<ipython-input-2-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-2-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [6]:
df2.head()

,Home Team,Home Team HT,Home Team FT,Away Team,Away Team HT,Away Team FT,date,time,Total HT Goals,Total FT Goals,FT-HT Goals
0,Crvena zvezda,1,2,Trabzonspor,1,1,2022-10-27,22:00,2,3,1
1,Ferencvaros,0,1,Monaco,1,1,2022-10-27,22:00,1,2,1
2,Freiburg,0,1,Olympiacos Piraeus,1,1,2022-10-27,22:00,1,2,1
3,HJK,0,1,AS Roma,1,2,2022-10-27,22:00,1,3,2
4,Manchester Utd,1,3,Sheriff Tiraspol,0,0,2022-10-27,22:00,1,3,2


In [3]:
#Croatia
"https://www.livesport.com/en/soccer/croatia/druga-nl/results/"
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/croatia/druga-nl/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-3-10cb8f41f39c>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['11.11. 16:00', 'Opatija', '(0)', '1', 'Zrinski Jurjevac', '(0)', '2'], ['11.11. 16:00', 'Sesvete', '(0)', '0', 'Hrvace', '(2)', '3']]


In [4]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)


In [5]:
df2=clean_data(df)

df2.to_csv('croatia.csv',index=False)

<ipython-input-2-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-2-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [16]:
#seric C
"https://www.livesport.com/en/soccer/italy/serie-c-group-a/results/"
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/italy/serie-c-group-a/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-16-ec0431457f11>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['12.11. 16:30', 'AlbinoLeffe', '(1)', '1', 'Sangiuliano City', '(0)', '1'], ['12.11. 16:30', 'Arzignano', '(0)', '0', 'Mantova', '(1)', '1']]


In [17]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)


In [18]:
df2=clean_data(df)

df2.to_csv('serie c.csv',index=False)

<ipython-input-2-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-2-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [19]:
#ligue 2
"https://www.livesport.com/en/soccer/france/ligue-2/results/"
#english premier league
#putting an object to have the list
element_list=[]

driver=webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.livesport.com/en/soccer/france/ligue-2/results/")

date=driver.find_elements(By.CLASS_NAME,"event__time")
home_team=driver.find_elements(By.CLASS_NAME,"event__participant--home")
home_half=driver.find_elements(By.CLASS_NAME,"event__part--home.event__part--1")
home_full=driver.find_elements(By.CLASS_NAME,"event__score--home")
away_team=driver.find_elements(By.CLASS_NAME,"event__participant--away")
away_half=driver.find_elements(By.CLASS_NAME,"event__part--away.event__part--1")
away_full=driver.find_elements(By.CLASS_NAME,"event__score--away")


for i in range(len(date)):
    element_list.append([date[i].text,home_team[i].text,home_half[i].text,home_full[i].text,
                        away_team[i].text,away_half[i].text,away_full[i].text])
print(element_list[:2])

#closing the driver
driver.close()

<ipython-input-19-81a4bda32b86>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(ChromeDriverManager().install())


[['12.11. 17:00', 'St Etienne', '(0)', '0', 'Rodez', '(0)', '2'], ['07.11. 22:45', 'Metz', '(2)', '3', 'St Etienne', '(1)', '2']]


In [20]:
#making the element list a data frame
import pandas as pd 
# List1 
df = pd.DataFrame(element_list, columns =['Date','Home Team','Home Team HT',
                                          'Home Team FT','Away Team',
                                          'Away Team HT','Away Team FT'],
                  dtype = float)


In [21]:
df2=clean_data(df)

df2.to_csv('ligue 2.csv',index=False)

<ipython-input-2-9d32f0990916>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Home Team HT']=df['Home Team HT'].str.replace(')','')
<ipython-input-2-9d32f0990916>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Away Team HT']=df['Away Team HT'].str.replace('(','')
<ipython-input-2-9d32f0990916>:8: FutureWarning: The default value of regex will change from True to False in a fut

In [25]:
df2.head()

,Home Team,Home Team HT,Home Team FT,Away Team,Away Team HT,Away Team FT,date,time,Total HT Goals,Total FT Goals,FT-HT Goals
0,B. Monchengladbach,2,3,Stuttgart,0,1,2022-01-14,19:30,2,4,2
1,Bochum,0,1,Werder Bremen,0,1,2022-01-14,17:30,0,2,2
2,Darmstadt,0,0,Dortmund,1,3,2022-01-13,20:30,1,3,2
3,Augsburg,0,0,Bayer Leverkusen,0,1,2022-01-13,17:30,0,1,1
4,FC Koln,1,1,Heidenheim,0,1,2022-01-13,17:30,1,2,1


In [26]:
#creating a function to predict the games
#importing libraries
import pandas as pd
from sklearn.linear_model import LinearRegression

In [37]:
def predict_total_goals(df):
    
    #create list to store predictions
    predictions = []
    
    #iterate through each row in the dataframe
    for index, row in df.iterrows():
        
        home_team=row['Home Team']
        away_team=row['Away Team']
        ht_score_home=row['Home Team HT']
        ht_score_away=row['Away Team HT']
        
    # Filter the dataset based on the teams playing
    team_data = df2[(df2['Home Team'] == home_team) & (df2['Away Team'] == away_team)]

    # Filter data for the Home Team's previous games
    home_team_data = team_data[team_data['Home Team'] == home_team]

    # Filter data for the Away Team's previous games
    away_team_data = team_data[team_data['Away Team'] == away_team]

    # Train a linear regression model for Home Team
    model_home = LinearRegression()
    model_home.fit(home_team_data[['Home Team HT']], home_team_data['Home Team FT'])

    # Train a linear regression model for Away Team
    model_away = LinearRegression()
    model_away.fit(away_team_data[['Away Team HT']], away_team_data['Away Team FT'])

    # Use the trained models to predict Full Time scores based on Half Time scores
    predicted_ft_home = model_home.predict([[ht_score_home]])[0]
    predicted_ft_away = model_away.predict([[ht_score_away]])[0]

    # Calculate the total predicted goals
    total_goals_prediction = predicted_ft_home + predicted_ft_away
    
    #append the predictions to the list
    predictions.append(total_goals_prediction)
    
    #add the predictions as a new column to the dataframe
    df['Predicted Total Goals'] = predictions

    #return total_goals_prediction

In [29]:
#sample data set with half time score for these teams
#drop some columns from the data we have
to_drop=['Home Team FT','Away Team FT','date','time','Total HT Goals',
        'Total FT Goals','FT-HT Goals']
df3_prediction=df2.drop(columns=to_drop)
df3_prediction.head()

,Home Team,Home Team HT,Away Team,Away Team HT
0,B. Monchengladbach,2,Stuttgart,0
1,Bochum,0,Werder Bremen,0
2,Darmstadt,0,Dortmund,1
3,Augsburg,0,Bayer Leverkusen,0
4,FC Koln,1,Heidenheim,0


In [50]:
df3_prediction_few= df3_prediction.tail(5)
df3_prediction_few.head()

,Home Team,Home Team HT,Away Team,Away Team HT
104,Mainz,0,Bayer Leverkusen,1
105,Wolfsburg,1,Eintracht Frankfurt,0
106,Hoffenheim,1,Dortmund,2
107,Eintracht Frankfurt,0,Freiburg,0
108,Bayer Leverkusen,1,Heidenheim,0


In [38]:
# Example usage:
# Assuming you have a DataFrame called 'your_dataset' with the necessary columns
# Replace 'your_dataset' with the actual variable name of your dataset
predicted_goals = predict_total_goals(df3_prediction_few)
#print(f'Predicted Total Goals: {predicted_goals:.2f}')

C:\Users\John\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\John\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: Length of values (1) does not match length of index (5)

In [44]:
def predict_total_goals(df, results_df):
    predictions = []

    for index, row in df.iterrows():
        home_team = row['Home Team']
        away_team = row['Away Team']
        ht_score_home = row['Home Team HT']
        ht_score_away = row['Away Team HT']

        # Assuming 'results_df' is your dataset with historical full-time results
        home_team_results = results_df[results_df['Home Team'] == home_team]
        away_team_results = results_df[results_df['Away Team'] == away_team]

        # Find the maximum number of goals each team has scored after half time
        max_goals_home = home_team_results['FT-HT_Home'].max()  # Replace 'FT Total Score' with the actual column name
        max_goals_away = away_team_results['FT-HT_Away'].max()  # Replace 'FT Total Score' with the actual column name

        # Rule-based predictions
        home_team_prediction = ht_score_home + max_goals_home  # Example rule: +2 goals after halftime if 1 goal at HT
        away_team_prediction = ht_score_away + max_goals_away  # Example rule: +1 goal after halftime if 0 goals at HT

        # Ensure that the total predicted goals don't exceed the team's historical maximum
        total_goals_prediction = home_team_prediction + away_team_prediction

        predictions.append(total_goals_prediction)

    # Add the predictions as a new column to the DataFrame
    df['Predicted Total Goals'] = predictions

# Example usage:
# Assuming you have a DataFrame called 'your_dataset' with the necessary columns
# Replace 'your_dataset' with the actual variable name of your dataset

# Assuming 'results_df' is your dataset with historical full-time results
# Replace 'results_df' with the actual variable name of your historical results dataset


In [51]:

# Call the function to add the predicted column
predictions=predict_total_goals(df3_prediction_few, df2)
df3_prediction_few.head()

<ipython-input-44-26bd1404f08e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Predicted Total Goals'] = predictions


,Home Team,Home Team HT,Away Team,Away Team HT,Predicted Total Goals
104,Mainz,0,Bayer Leverkusen,1,5
105,Wolfsburg,1,Eintracht Frankfurt,0,4
106,Hoffenheim,1,Dortmund,2,7
107,Eintracht Frankfurt,0,Freiburg,0,3
108,Bayer Leverkusen,1,Heidenheim,0,6


In [49]:
df3_prediction_few.head()

,Home Team,Home Team HT,Away Team,Away Team HT,Predicted Total Goals
0,B. Monchengladbach,2,Stuttgart,0,6
1,Bochum,0,Werder Bremen,0,4
2,Darmstadt,0,Dortmund,1,5
3,Augsburg,0,Bayer Leverkusen,0,4
4,FC Koln,1,Heidenheim,0,5


In [ ]:
df['Predicted Total Goals'] = predictions

In [42]:
df2['FT-HT_Home']=df2['Home Team FT'] - df2['Home Team HT']
df2['FT-HT_Away']=df2['Away Team FT'] - df2['Away Team HT']

In [43]:
df2.head()

,Home Team,Home Team HT,Home Team FT,Away Team,Away Team HT,Away Team FT,date,time,Total HT Goals,Total FT Goals,FT-HT Goals,FT-HT_Home,FT-HT_Away
0,B. Monchengladbach,2,3,Stuttgart,0,1,2022-01-14,19:30,2,4,2,1,1
1,Bochum,0,1,Werder Bremen,0,1,2022-01-14,17:30,0,2,2,1,1
2,Darmstadt,0,0,Dortmund,1,3,2022-01-13,20:30,1,3,2,0,2
3,Augsburg,0,0,Bayer Leverkusen,0,1,2022-01-13,17:30,0,1,1,0,1
4,FC Koln,1,1,Heidenheim,0,1,2022-01-13,17:30,1,2,1,0,1
